# Low Mach approximation Navier-Stokes element formulation and automatic differentiation
**Rubén Zorrilla**
UPC BarcelonaTech, CIMNE

## Governing equations
This formulation comes from a simplification of the fully compressible Navier-Stokes equation and is suitable for the resolution of low speed strongly thermally coupled flows.
Specifically, the formulation indroduces the low Mach limit simplification, which allows to split the total pressure into two parts, a thermodynamic part $p_{th}$ that is uniform in space and a mechanical (hydrodynamic) one $p$.
It is important to note that $p$ is several orders of magnitude smaller than $p_{th}$ and is therefore omitted from the state and energy equations.
In other words, the compressibility of the fluid comes from the thermal effects only.
Hence, the problem to be considered is finding a velocity field $\bm{u}$, mechanical pressure $p$, temperature field $T$ and thermodynamic pressure $p_{th}$ such that
\begin{equation*}
\begin{aligned}
    \frac{\partial \rho}{\partial t} + \nabla\cdot\left(\rho\bm{u}\right) &= 0 \\
    \rho\frac{\partial\bm{u}}{\partial t} + \rho\bm{u}\cdot\nabla\bm{u} - \nabla\cdot\boldsymbol{\tau} + \nabla p + \sigma(\bm{u} - \bm{u}_{s}) + \sigma\left(\bm{u} - \bm{u}_{s}\right) &= \rho\bm{g} \\
    \rho c_{p}\frac{\partial T}{\partial t} + \rho c_{p}\bm{u}\cdot\nabla T - \nabla\cdot\left(\kappa\nabla T\right) - \alpha T\frac{d p_{th}}{dt} &= Q
\end{aligned}
\; ,
\end{equation*}
being $c_{p}$ the specific heat coefficient at constant pressure, $\kappa$ the thermal conductivity and $\alpha$ the thermal expansion coefficient.
The system must be closed by a state equation relating $\rho$, $T$ and $p_{th}$.
Assuming an ideal gas behavior, the state equation is $\rho=p_{th}/RT$, with $R$ being the specific gas constant that relates heat capacity coefficients at constant pressure and constant volume as $R = c_{p} - c_{v}$.
Introducing the heat capacity ratio $\gamma = c_{p} / c_{v}$ allows to alternatively write the state equation for ideal gases as
\begin{equation*}
    \rho = \frac{\gamma p_{th}}{c_{p}(\gamma - 1)T}\; ,
\end{equation*}
something that at the same time makes possible to obtain the following magnitudes as
\begin{equation*}
    \frac{\partial \rho}{\partial t} = \frac{\gamma}{c_{p}(\gamma - 1)}\left(\frac{1}{T}\frac{d p_{th}}{dt}-\frac{p_{th}}{T^{2}}\frac{\partial T}{\partial t}\right)\; ,
\end{equation*}

\begin{equation*}
    \nabla \rho = -\frac{\gamma p_{th}}{c_{p}(\gamma - 1)T^{2}}\nabla T
\end{equation*}
and
\begin{equation*}
    \alpha = \left.-\frac{1}{\rho}\frac{\partial \rho}{\partial T}\right|_{p} = \frac{1}{T}\;.
\end{equation*}

The determination of the thermodynamic pressure depends on the nature of the flow, which can be roughly divided in \emph{open} and \emph{closed} flows.
For open flows (\ie flows with Neumann boundary condition) $p_{th}$ is given at each time step by the boundary conditions.
For closed flows (\ie flows without Neumann boundary condition) of ideal gases with neither inflow nor outflow conditions $p_{th}$ can be obtained from the $T$ field as
\begin{equation*}
    p_{th} = p_{{th}_{0}}\frac{\int_{\Omega}\frac{1}{T_{0}}}{\int_{\Omega}\frac{1}{T}}\; ,
\end{equation*}
where the subscript $0$ indicates the initial magnitudes at $t=0$ and $\Omega$ the problem domain.

<!-- ## Variational form
The variational form (i.e. the functional to be automatically differenctiated) is obtained by multiplying previous equations by the corresponding test functions $\mathbf{v}$, $q$ and $w$. After integration by parts, this yields

\begin{align*}
\left(w_{x},\rho g_{x}\right)_{\Omega} - & \left(w_{x},\rho\frac{\partial u_{x}}{\partial t}\right)_{\Omega} - \left(w_{x},\rho a_{y}\frac{\partial u_{x}}{\partial y}\right)_{\Omega} - \left(w_{x},\rho a_{x}\frac{\partial u_{x}}{\partial x}\right)_{\Omega} + \left(\frac{\partial w_{x}}{\partial x}, p\right)_{\Omega} \\
- & \left(\frac{\partial w_{x}}{\partial y}, \mu\frac{\partial u_{x}}{\partial y}\right)_{\Omega} - \left(\frac{\partial w_{x}}{\partial x}, \mu\frac{\partial u_{x}}{\partial x}\right)_{\Omega} + \langle w_{x}, \mu\left(\frac{\partial u_{x}}{\partial y}+\frac{\partial u_{x}}{\partial x}\right)n_{x} - p n_{x} \rangle_{\Gamma} \\
+ & \left(w_{y},\rho g_{y}\right)_{\Omega} - \left(w_{y},\rho\frac{\partial u_{y}}{\partial t}\right)_{\Omega} - \left(w_{y},\rho a_{y}\frac{\partial u_{y}}{\partial y}\right)_{\Omega} - \left(w_{y},\rho a_{x}\frac{\partial u_{y}}{\partial x}\right)_{\Omega}
+ \left(\frac{w_{y}}{y} + \frac{\partial w_{y}}{\partial y}, p\right)_{\Omega} \\
- & \left(\frac{\partial w_{y}}{\partial y}, \mu\frac{\partial u_{y}}{\partial y}\right)_{\Omega} - \left(\frac{\partial w_{y}}{\partial x}, \mu\frac{\partial u_{y}}{\partial x}\right)_{\Omega} + \langle w_{y}, \mu\left(\frac{\partial u_{y}}{\partial y}+\frac{\partial u_{y}}{\partial x}\right)n_{y} - p n_{y} \rangle_{\Gamma} \\
- & \left(q,\frac{u_{y}}{y}\right)_{\Omega} - \left(q,\frac{\partial u_{y}}{\partial y}\right)_{\Omega} - \left(q, \frac{\partial u_{x}}{\partial x}\right)_{\Omega} = 0\; ,
\end{align*}
being $\mathbf{a}$ the convective velocity, which computed from the linearised previous iteration velocity $\mathbf{u}^{k-1}$ and the mesh velocity $\mathbf{u}_{M}$ as
\begin{equation*}
\mathbf{a} = \mathbf{u}^{k-1} - \mathbf{u}_{M}\; .
\end{equation*}

Note that the cylindrical component of the divergence operator applied to the viscous stress term (see the strong form above) no longer appears in the variational form as we get rid of it by integrating by parts.
However, it appears in the test function divergence of the pressure term.

## Variational MultiScales stabilization
The scales separation reads

\begin{align*}
\mathbf{u} & = \mathbf{u_{h}}+\mathbf{u_{s}}\\
p & = p_{h}+p_{s}
\end{align*}

being $\mathbf{u}_{h}$ and $p_{h}$ the finite element scales and $\mathbf{u}_{s}$ and $p_{s}$ the subgrid scales, which are modelled from the finite element residuals as

\begin{align*}
u_{s,x} & = \tau_{1} R^{M}_{x}(u_{h,x},p_{h})\; , \\
u_{s,y} & = \tau_{1} R^{M}_{y}(u_{h,y},p_{h})\; , \\
p_{s} & = \tau_{2} R^{C}(\mathbf{u}_{h},p_{h})\; ,
\end{align*}

being $\tau_{1}$ and $\tau_{2}$ the stabilization parameters which in this case equal
\begin{equation*}
    \tau_{1} = \left(\frac{\rho\tau_{d}}{\Delta t} + \frac{c_{1}\mu}{h^{2}} + \frac{c_{2}\rho\lVert\mathbf{a}\rVert}{h}\right)^{-1}
\end{equation*}
and
\begin{equation*}
    \tau_{2} = \mu+\frac{c_{2}\rho h\lVert\mathbf{a}\rVert}{c_{1}} \; ,
\end{equation*}
being $c_{1}$ and $c_{2}$ the user-definable stabilization constants, $\tau_{d}$ a dynamic coefficient ranging between 0 and 1 and $h$ the characteristic element size.

The finite element residuals $\mathbf{R}^{M}(\mathbf{u}_{h},p_{h}$ and $R^{C}(\mathbf{u}_{h},p_{h})$ are defined as
\begin{align*}
R^{M}_{x}(u_{h,x},p_{h}) = & \rho g_{x} - \rho\frac{\partial u_{h,x}}{\partial t} - \rho a_{h,y}\frac{\partial u_{h,x}}{\partial y} - \rho a_{h,x}\frac{\partial u_{h,x}}{\partial x} - \frac{\partial p_{h}}{\partial x} + \mu\left(\frac{1}{y}\frac{\partial u_{h,x}}{\partial y} + \frac{\partial^{2} u_{h,x}}{\partial y^{2}} + \frac{\partial^{2} u_{h,x}}{\partial x^{2}}\right)\; , \\
R^{M}_{y}(u_{h,y},p_{h}) = & \rho g_{y} - \rho\frac{\partial u_{h,y}}{\partial t} - \rho a_{h,y}\frac{\partial u_{h,y}}{\partial y} - \rho a_{h,x}\frac{\partial u_{h,y}}{\partial x} - \frac{\partial p_{h}}{\partial y} + \mu\left(- \frac{u_{h,y}}{y^{2}} + \frac{1}{y}\frac{\partial u_{h,y}}{\partial y} + \frac{\partial^{2} u_{h,y}}{\partial y^{2}} + \frac{\partial^{2} u_{h,y}}{\partial x^{2}}\right)\; , \\
R^{C}(\mathbf{u}_{h},p_{h}) = & -\frac{1}{y}u_{h,y} - \frac{\partial u_{h,y}}{\partial y} - \frac{\partial u_{h,x}}{\partial x} \; .
\end{align*}

Once the subscales are defined, one can derive the stabilization terms to be added to the functional above. Hence, after doing the required integration by parts to remove the subscale derivatives and dropping the high order terms (the use of linear elements is assumed), one obtains the stabilization functional
\begin{align*}
\left(\frac{\partial w_{x}}{\partial y}\rho a_{y} + w_{x}\rho\frac{\partial a_{y}}{\partial y}, u_{s,x}\right)_{\Omega} & + \left(\frac{\partial w_{x}}{\partial x}\rho a_{x} + w_{x}\rho\frac{\partial a_{x}}{\partial x}, u_{s,x}\right)_{\Omega} + \left(\frac{\partial w_{x}}{\partial x},p_{s}\right)_{\Omega} \\
& + \left(\frac{\partial w_{y}}{\partial y}\rho a_{y} + w_{y}\rho\frac{\partial a_{y}}{\partial y}, u_{s,y}\right)_{\Omega} + \left(\frac{\partial w_{y}}{\partial x}\rho a_{x} + w_{y}\rho\frac{\partial a_{x}}{\partial x}, u_{s,y}\right)_{\Omega} + \left(\frac{w_{y}}{y} + \frac{\partial w_{y}}{\partial y},p_{s}\right)_{\Omega} \\
& - \left(q,\frac{1}{y}u_{s,y}\right)_{\Omega} + \left(\frac{\partial q}{\partial y},u_{s,y}\right)_{\Omega} + \left(\frac{\partial q}{\partial x},u_{s,x}\right)_{\Omega}
\end{align*}
in which the quasi-static subscales assumption has been considered. -->
<!-- 
## IMPLEMENTATION
The remaining implementation is similar to that of the weakly compressible Navier-Stokes element but with the added simplification of assuming a Newtonian constitutive model within the element.  --> -->
